In [1]:
import os
import re
from glob import glob

In [2]:
POSSIBLE_LABELS = 'yes no up down left right on off stop go silence unknown'.split()
id2name = {i: name for i, name in enumerate(POSSIBLE_LABELS)}
name2id = {name: i for i, name in id2name.items()}

In [3]:
def load_data(data_dir):
    """ Return 2 lists of tuples:
    [(class_id, user_id, path), ...] for train
    [(class_id, user_id, path), ...] for validation
    """
    # Just a simple regexp for paths with three groups:
    # prefix, label, user_id
    pattern = re.compile("(.+\/)?(\w+)\/([^_]+)_.+wav")
    all_files = glob(os.path.join(data_dir, 'train/audio/*/*wav'))

    with open(os.path.join(data_dir, 'train/validation_list.txt'), 'r') as fin:
        validation_files = fin.readlines()
    valset = set()
    for entry in validation_files:
        r = re.match(pattern, entry)
        if r:
            valset.add(r.group(3))

    possible = set(POSSIBLE_LABELS)
    train, val = [], []
    for entry in all_files:
        r = re.match(pattern, entry)
        if r:
            label, uid = r.group(2), r.group(3)
            if label == '_background_noise_':
                label = 'silence'
            if label not in possible:
                label = 'unknown'

            label_id = name2id[label]

            sample = (label_id, uid, entry)
            if uid in valset:
                val.append(sample)
            else:
                train.append(sample)

    print('There are {} train and {} val samples'.format(len(train), len(val)))
    return train, val

In [4]:
trainset, valset = load_data('./data/')

There are 57929 train and 6798 val samples


In [5]:
import numpy as np
from scipy.io import wavfile

In [6]:
def data_generator(data, params, mode='train'):
    def generator():
        if mode == 'train':
            np.random.shuffle(data)
        # Feel free to add any augmentation
        for (label_id, uid, fname) in data:
            try:
                _, wav = wavfile.read(fname)
                wav = wav.astype(np.float32) / np.iinfo(np.int16).max

                L = 16000  # be aware, some files are shorter than 1 sec!
                if len(wav) < L:
                    continue
                # let's generate more silence!
                samples_per_file = 1 if label_id != name2id['silence'] else 20
                for _ in range(samples_per_file):
                    if len(wav) > L:
                        beg = np.random.randint(0, len(wav) - L)
                    else:
                        beg = 0
                    yield dict(
                        target=np.int32(label_id),
                        wav=wav[beg: beg + L],
                    )
            except Exception as err:
                print(err, label_id, uid, fname)

    return generator

In [7]:
import tensorflow as tf
from tensorflow.contrib import layers

In [9]:
def baseline(x, params, is_training):
    x = layers.batch_norm(x, is_training=is_training)
    for i in range(4):
        x = layers.conv2d(
            x, 16 * (2 ** i), 3, 1,
            activation_fn=tf.nn.elu,
            normalizer_fn=layers.batch_norm if params.use_batch_norm else None,
            normalizer_params={'is_training': is_training}
        )
        x = layers.max_pool2d(x, 2, 2)

    # just take two kind of pooling and then mix them, why not :)
    mpool = tf.reduce_max(x, axis=[1, 2], keep_dims=True)
    apool = tf.reduce_mean(x, axis=[1, 2], keep_dims=True)

    x = 0.5 * (mpool + apool)
    # we can use conv2d 1x1 instead of dense
    x = layers.conv2d(x, 128, 1, 1, activation_fn=tf.nn.elu)
    x = tf.nn.dropout(x, keep_prob=params.keep_prob if is_training else 1.0)
    
    # again conv2d 1x1 instead of dense layer
    logits = layers.conv2d(x, params.num_classes, 1, 1, activation_fn=None)
    return tf.squeeze(logits, [1, 2])

In [10]:
from tensorflow.contrib import signal

In [11]:
# features is a dict with keys: tensors from our datagenerator
# labels also were in features, but excluded in generator_input_fn by target_key

def model_handler(features, labels, mode, params, config):
    # Im really like to use make_template instead of variable_scopes and re-usage
    extractor = tf.make_template(
        'extractor', baseline,
        create_scope_now_=True,
    )
    # wav is a waveform signal with shape (16000, )
    wav = features['wav']
    # we want to compute spectograms by means of short time fourier transform:
    specgram = signal.stft(
        wav,
        400,  # 16000 [samples per second] * 0.025 [s] -- default stft window frame
        160,  # 16000 * 0.010 -- default stride
    )
    # specgram is a complex tensor, so split it into abs and phase parts:
    phase = tf.angle(specgram) / np.pi
    # log(1 + abs) is a default transformation for energy units
    amp = tf.log1p(tf.abs(specgram))
    
    x = tf.stack([amp, phase], axis=3) # shape is [bs, time, freq_bins, 2]
    x = tf.to_float(x)  # we want to have float32, not float64

    logits = extractor(x, params, mode == tf.estimator.ModeKeys.TRAIN)

    if mode == tf.estimator.ModeKeys.TRAIN:
        loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
        # some lr tuner, you could use move interesting functions
        def learning_rate_decay_fn(learning_rate, global_step):
            return tf.train.exponential_decay(
                learning_rate, global_step, decay_steps=10000, decay_rate=0.99)

        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=params.learning_rate,
            optimizer=lambda lr: tf.train.MomentumOptimizer(lr, 0.9, use_nesterov=True),
            learning_rate_decay_fn=learning_rate_decay_fn,
            clip_gradients=params.clip_gradients,
            variables=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

        specs = dict(
            mode=mode,
            loss=loss,
            train_op=train_op,
        )

    if mode == tf.estimator.ModeKeys.EVAL:
        prediction = tf.argmax(logits, axis=-1)
        acc, acc_op = tf.metrics.mean_per_class_accuracy(
            labels, prediction, params.num_classes)
        loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
        specs = dict(
            mode=mode,
            loss=loss,
            eval_metric_ops=dict(
                acc=(acc, acc_op),
            )
        )

    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'label': tf.argmax(logits, axis=-1),  # for probability just take tf.nn.softmax()
            'sample': features['sample'], # it's a hack for simplicity
        }
        specs = dict(
            mode=mode,
            predictions=predictions,
        )
    return tf.estimator.EstimatorSpec(**specs)


def create_model(config=None, hparams=None):
    return tf.estimator.Estimator(
        model_fn=model_handler,
        config=config,
        params=hparams)

In [12]:
params=dict(
    seed=2018,
    batch_size=64,
    keep_prob=0.5,
    learning_rate=1e-3,
    clip_gradients=15.0,
    use_batch_norm=True,
    num_classes=len(POSSIBLE_LABELS),
)

hparams = tf.contrib.training.HParams(**params)
os.makedirs(os.path.join('./out/', 'eval'), exist_ok=True)
model_dir = './out/'

run_config = tf.contrib.learn.RunConfig(model_dir=model_dir)

In [13]:
# it's a magic function :)
from tensorflow.contrib.learn.python.learn.learn_io.generator_io import generator_input_fn

In [14]:
train_input_fn = generator_input_fn(
    x=data_generator(trainset, hparams, 'train'),
    target_key='target',  # you could leave target_key in features, so labels in model_handler will be empty
    batch_size=hparams.batch_size, shuffle=True, num_epochs=None,
    queue_capacity=3 * hparams.batch_size + 10, num_threads=1,
)

In [15]:
val_input_fn = generator_input_fn(
    x=data_generator(valset, hparams, 'val'),
    target_key='target',
    batch_size=hparams.batch_size, shuffle=True, num_epochs=None,
    queue_capacity=3 * hparams.batch_size + 10, num_threads=1,
)

In [16]:
def _create_my_experiment(run_config, hparams):
    exp = tf.contrib.learn.Experiment(
        estimator=create_model(config=run_config, hparams=hparams),
        train_input_fn=train_input_fn,
        eval_input_fn=val_input_fn,
        train_steps=10000, # just randomly selected params
        eval_steps=200,  # read source code for steps-epochs ariphmetics
        train_steps_per_iteration=1000,
    )
    return exp

In [17]:
tf.contrib.learn.learn_runner.run(
    experiment_fn=_create_my_experiment,
    run_config=run_config,
    schedule="continuous_train_and_eval",
    hparams=hparams)

INFO:tensorflow:Using config: {'_session_config': None, '_task_type': None, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_log_step_count_steps': 100, '_is_chief': True, '_save_summary_steps': 100, '_model_dir': './out/', '_tf_random_seed': None, '_master': '', '_task_id': 0, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5757c52fd0>, '_save_checkpoints_steps': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_num_worker_replicas': 0, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_environment': 'local'}
INFO:tensorflow:Training model for 1000 steps
Instructions for updating:
Please switch to tf.train.get_global_step
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./out/model.ckpt.
INFO:tensorflow:step = 1, loss = 5.55493
INFO:tensorflow:global_step/sec: 1.79921
INFO:tensorflow:step = 101, loss = 1.45167 (55.581 sec)
INFO:ten

/home/sainath/anaconda3/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


INFO:tensorflow:global_step/sec: 1.77677
INFO:tensorflow:step = 301, loss = 1.26554 (56.281 sec)
INFO:tensorflow:global_step/sec: 1.78202
INFO:tensorflow:step = 401, loss = 1.01339 (56.113 sec)
INFO:tensorflow:global_step/sec: 1.75679
INFO:tensorflow:step = 501, loss = 1.20635 (56.922 sec)
INFO:tensorflow:global_step/sec: 1.74688
INFO:tensorflow:step = 601, loss = 1.11947 (57.248 sec)
INFO:tensorflow:global_step/sec: 1.75073
INFO:tensorflow:step = 701, loss = 1.04894 (57.118 sec)
INFO:tensorflow:global_step/sec: 1.7161
INFO:tensorflow:step = 801, loss = 0.791654 (58.269 sec)
INFO:tensorflow:global_step/sec: 5.60535
INFO:tensorflow:step = 901, loss = 0.630179 (17.840 sec)
INFO:tensorflow:Saving checkpoints for 1000 into ./out/model.ckpt.
INFO:tensorflow:Loss for final step: 0.560275.
INFO:tensorflow:Evaluating model now.
INFO:tensorflow:Starting evaluation at 2017-12-27-10:10:16
INFO:tensorflow:Restoring parameters from ./out/model.ckpt-1000
INFO:tensorflow:Evaluation [1/200]
INFO:tenso

INFO:tensorflow:Evaluation [200/200]
INFO:tensorflow:Finished evaluation at 2017-12-27-10:10:57
INFO:tensorflow:Saving dict for global step 1000: acc = 0.088442, global_step = 1000, loss = 1.17379
INFO:tensorflow:Training model for 1000 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./out/model.ckpt-1000
INFO:tensorflow:Saving checkpoints for 1001 into ./out/model.ckpt.
INFO:tensorflow:step = 1001, loss = 0.791772
INFO:tensorflow:global_step/sec: 8.76189
INFO:tensorflow:step = 1101, loss = 0.710869 (11.418 sec)
INFO:tensorflow:global_step/sec: 9.02599
INFO:tensorflow:step = 1201, loss = 0.717979 (11.077 sec)
INFO:tensorflow:global_step/sec: 9.23964
INFO:tensorflow:step = 1301, loss = 0.483907 (10.823 sec)
INFO:tensorflow:global_step/sec: 9.26142
INFO:tensorflow:step = 1401, loss = 0.665792 (10.797 sec)
INFO:tensorflow:global_step/sec: 9.24335
INFO:tensorflow:step = 1501, loss = 0.668086 (10.818 sec)
INFO:tensorflow:global_step/sec: 9.2337
IN

INFO:tensorflow:Evaluation [182/200]
INFO:tensorflow:Evaluation [183/200]
INFO:tensorflow:Evaluation [184/200]
INFO:tensorflow:Evaluation [185/200]
INFO:tensorflow:Evaluation [186/200]
INFO:tensorflow:Evaluation [187/200]
INFO:tensorflow:Evaluation [188/200]
INFO:tensorflow:Evaluation [189/200]
INFO:tensorflow:Evaluation [190/200]
INFO:tensorflow:Evaluation [191/200]
INFO:tensorflow:Evaluation [192/200]
INFO:tensorflow:Evaluation [193/200]
INFO:tensorflow:Evaluation [194/200]
INFO:tensorflow:Evaluation [195/200]
INFO:tensorflow:Evaluation [196/200]
INFO:tensorflow:Evaluation [197/200]
INFO:tensorflow:Evaluation [198/200]
INFO:tensorflow:Evaluation [199/200]
INFO:tensorflow:Evaluation [200/200]
INFO:tensorflow:Finished evaluation at 2017-12-27-10:13:03
INFO:tensorflow:Saving dict for global step 2000: acc = 0.251046, global_step = 2000, loss = 0.826048
INFO:tensorflow:Training model for 1000 steps
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./ou

...

INFO:tensorflow:Evaluation [92/200]
INFO:tensorflow:Evaluation [93/200]
INFO:tensorflow:Evaluation [94/200]
INFO:tensorflow:Evaluation [95/200]
INFO:tensorflow:Evaluation [96/200]
INFO:tensorflow:Evaluation [97/200]
INFO:tensorflow:Evaluation [98/200]
INFO:tensorflow:Evaluation [99/200]
INFO:tensorflow:Evaluation [100/200]
INFO:tensorflow:Evaluation [101/200]
INFO:tensorflow:Evaluation [102/200]
INFO:tensorflow:Evaluation [103/200]
INFO:tensorflow:Evaluation [104/200]
INFO:tensorflow:Evaluation [105/200]
INFO:tensorflow:Evaluation [106/200]
INFO:tensorflow:Evaluation [107/200]
INFO:tensorflow:Evaluation [108/200]
INFO:tensorflow:Evaluation [109/200]
INFO:tensorflow:Evaluation [110/200]
INFO:tensorflow:Evaluation [111/200]
INFO:tensorflow:Evaluation [112/200]
INFO:tensorflow:Evaluation [113/200]
INFO:tensorflow:Evaluation [114/200]
INFO:tensorflow:Evaluation [115/200]
INFO:tensorflow:Evaluation [116/200]
INFO:tensorflow:Evaluation [117/200]
INFO:tensorflow:Evaluation [118/200]
INFO:tens

INFO:tensorflow:Evaluation [73/200]
INFO:tensorflow:Evaluation [74/200]
INFO:tensorflow:Evaluation [75/200]
INFO:tensorflow:Evaluation [76/200]
INFO:tensorflow:Evaluation [77/200]
INFO:tensorflow:Evaluation [78/200]
INFO:tensorflow:Evaluation [79/200]
INFO:tensorflow:Evaluation [80/200]
INFO:tensorflow:Evaluation [81/200]
INFO:tensorflow:Evaluation [82/200]
INFO:tensorflow:Evaluation [83/200]
INFO:tensorflow:Evaluation [84/200]
INFO:tensorflow:Evaluation [85/200]
INFO:tensorflow:Evaluation [86/200]
INFO:tensorflow:Evaluation [87/200]
INFO:tensorflow:Evaluation [88/200]
INFO:tensorflow:Evaluation [89/200]
INFO:tensorflow:Evaluation [90/200]
INFO:tensorflow:Evaluation [91/200]
INFO:tensorflow:Evaluation [92/200]
INFO:tensorflow:Evaluation [93/200]
INFO:tensorflow:Evaluation [94/200]
INFO:tensorflow:Evaluation [95/200]
INFO:tensorflow:Evaluation [96/200]
INFO:tensorflow:Evaluation [97/200]
INFO:tensorflow:Evaluation [98/200]
INFO:tensorflow:Evaluation [99/200]
INFO:tensorflow:Evaluation [

INFO:tensorflow:Evaluation [54/200]
INFO:tensorflow:Evaluation [55/200]
INFO:tensorflow:Evaluation [56/200]
INFO:tensorflow:Evaluation [57/200]
INFO:tensorflow:Evaluation [58/200]
INFO:tensorflow:Evaluation [59/200]
INFO:tensorflow:Evaluation [60/200]
INFO:tensorflow:Evaluation [61/200]
INFO:tensorflow:Evaluation [62/200]
INFO:tensorflow:Evaluation [63/200]
INFO:tensorflow:Evaluation [64/200]
INFO:tensorflow:Evaluation [65/200]
INFO:tensorflow:Evaluation [66/200]
INFO:tensorflow:Evaluation [67/200]
INFO:tensorflow:Evaluation [68/200]
INFO:tensorflow:Evaluation [69/200]
INFO:tensorflow:Evaluation [70/200]
INFO:tensorflow:Evaluation [71/200]
INFO:tensorflow:Evaluation [72/200]
INFO:tensorflow:Evaluation [73/200]
INFO:tensorflow:Evaluation [74/200]
INFO:tensorflow:Evaluation [75/200]
INFO:tensorflow:Evaluation [76/200]
INFO:tensorflow:Evaluation [77/200]
INFO:tensorflow:Evaluation [78/200]
INFO:tensorflow:Evaluation [79/200]
INFO:tensorflow:Evaluation [80/200]
INFO:tensorflow:Evaluation [

INFO:tensorflow:Evaluation [35/200]
INFO:tensorflow:Evaluation [36/200]
INFO:tensorflow:Evaluation [37/200]
INFO:tensorflow:Evaluation [38/200]
INFO:tensorflow:Evaluation [39/200]
INFO:tensorflow:Evaluation [40/200]
INFO:tensorflow:Evaluation [41/200]
INFO:tensorflow:Evaluation [42/200]
INFO:tensorflow:Evaluation [43/200]
INFO:tensorflow:Evaluation [44/200]
INFO:tensorflow:Evaluation [45/200]
INFO:tensorflow:Evaluation [46/200]
INFO:tensorflow:Evaluation [47/200]
INFO:tensorflow:Evaluation [48/200]
INFO:tensorflow:Evaluation [49/200]
INFO:tensorflow:Evaluation [50/200]
INFO:tensorflow:Evaluation [51/200]
INFO:tensorflow:Evaluation [52/200]
INFO:tensorflow:Evaluation [53/200]
INFO:tensorflow:Evaluation [54/200]
INFO:tensorflow:Evaluation [55/200]
INFO:tensorflow:Evaluation [56/200]
INFO:tensorflow:Evaluation [57/200]
INFO:tensorflow:Evaluation [58/200]
INFO:tensorflow:Evaluation [59/200]
INFO:tensorflow:Evaluation [60/200]
INFO:tensorflow:Evaluation [61/200]
INFO:tensorflow:Evaluation [

({'acc': 0.78457069, 'global_step': 10000, 'loss': 0.27100348}, [])

In [18]:
from tqdm import tqdm

In [19]:
# now we want to predict!
paths = glob(os.path.join('./data/', 'test/audio/*wav'))

In [20]:
def test_data_generator(data):
    def generator():
        for path in data:
            _, wav = wavfile.read(path)
            wav = wav.astype(np.float32) / np.iinfo(np.int16).max
            fname = os.path.basename(path)
            yield dict(
                sample=np.string_(fname),
                wav=wav,
            )

    return generator

In [21]:
test_input_fn = generator_input_fn(
    x=test_data_generator(paths),
    batch_size=hparams.batch_size, 
    shuffle=False, 
    num_epochs=1,
    queue_capacity= 10 * hparams.batch_size, 
    num_threads=1,
)

In [22]:
model = create_model(config=run_config, hparams=hparams)
it = model.predict(input_fn=test_input_fn)

INFO:tensorflow:Using config: {'_session_config': None, '_task_type': None, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_log_step_count_steps': 100, '_is_chief': True, '_save_summary_steps': 100, '_model_dir': './out/', '_tf_random_seed': None, '_master': '', '_task_id': 0, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5757c52fd0>, '_save_checkpoints_steps': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_num_worker_replicas': 0, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_environment': 'local'}


In [23]:
# last batch will contain padding, so remove duplicates
submission = dict()
for t in tqdm(it):
    fname, label = t['sample'].decode(), id2name[t['label']]
    submission[fname] = label

0it [00:00, ?it/s]

INFO:tensorflow:Restoring parameters from ./out/model.ckpt-10000


158560it [23:02, 114.71it/s]


In [24]:
with open(os.path.join(model_dir, 'submission1.csv'), 'w') as fout:
    fout.write('fname,label\n')
    for fname, label in submission.items():
        fout.write('{},{}\n'.format(fname, label))